# FACE MASK DETECTION
We detect users with mask or not based in dataset from:
 https://github.com/cabani/MaskedFace-Net 


In [1]:
from google.colab import drive
drive.mount("/content/drive/",force_remount=True)

Mounted at /content/drive/


In [2]:
# UTILS + PREPROCESSING
import os
import imutils
import numpy as np
import pandas as pd
import cv2 as cv2
import glob
import matplotlib.pyplot as plt
import skimage as sk
from skimage import transform
import copy
import torch
from tqdm.notebook import tqdm
import re
from PIL import Image
import skimage.exposure as exposure

!pip install mtcnn
!pip install facenet_pytorch
import mtcnn.mtcnn
from facenet_pytorch import MTCNN

main_path = '/content/drive/MyDrive/FaceMask/'

#TORCH config
name = 'PyTorch\nGPU'
torchdevice = 0
if torch.cuda.is_available():
    device = torch.device('cuda:{}'.format(torchdevice))
else:
    device = 'cpu'
print('{} available: {}'.format(name, torch.cuda.is_available()))

PyTorch
GPU available: True


## Collecting DATA + Preprocessing + Feature extraction
We do the following:
1. Collect the data
2. Resize the images
3. Denoise
4. Extract features
5. Create a dataframe with the data

In [3]:
def process_img(image_path):
    #parameter for size output szeXsze
    sze = 224
    #parameters for denoising
    h=10
    templateWindowSize=7
    searchWindowSize=21
    
    #read+gray+resize+denoising
    img = cv2.imread( image_path, cv2.IMREAD_GRAYSCALE )
    #img = cv2.cvtColor( img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize( img, (sze,sze) )
    img = cv2.fastNlMeansDenoising(img,None,h,templateWindowSize,searchWindowSize)
    return img

def feature_extraction(img):

    #parameters for Gaussian blur
    GBlur = 0
    SBlur = 1.3
    
    #parameters for Sobel Edge detection
    ksize_s = 3
    
    #parameters for Canny Edge detection
    threshold1_c = 100
    threshold2_c = 200
    
    
    #Gaussian Blur
    img_blur = cv2.GaussianBlur(img, (GBlur,GBlur), sigmaX=SBlur, sigmaY=SBlur )
    
    #Sobel Edge
    sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=ksize_s) # Sobel Edge Detection on the X axis
    sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=ksize_s) # Sobel Edge Detection on the Y axis
    sobelx2 = cv2.multiply(sobelx,sobelx)
    sobely2 = cv2.multiply(sobely,sobely)
    # add together and take square root
    sobel_magnitude = cv2.sqrt(sobelx2 + sobely2)
    # normalize to range 0 to 255 and clip negatives
    sobel_magnitude = exposure.rescale_intensity(sobel_magnitude, in_range='image', out_range=(0,255)).clip(0,255).astype(np.uint8)
        
    #Canny Edge
    canny = cv2.Canny(image=img_blur, threshold1=threshold1_c, threshold2=threshold1_c)
    
    #Laplacian of Gaussian
    lap = cv2.Laplacian(img_blur, ddepth=cv2.CV_64F, ksize=ksize_s)
    lap = cv2.convertScaleAbs(lap)
    
    return  {'LaofGau':lap, 'SobelEdgeXY':sobel_magnitude, 'CannyEdge':canny}

#For plotting image
def show( img ):
    plt.imshow( img, cmap='gray' )
    return

In [4]:
#detect faces with mtcnn
# we use this library after reading https://towardsdatascience.com/face-detection-models-which-to-use-and-why-d263e82c302c
# info in https://www.kaggle.com/timesler/guide-to-mtcnn-in-facenet-pytorch#Bounding-boxes-and-facial-landmarks
detector = mtcnn.mtcnn.MTCNN()
def ReturnLocalizedFace( img ):
    faces = detector.detect_faces( cv2.cvtColor( img, cv2.COLOR_GRAY2RGB ) )
    return faces

#for many faces
def ReturnLocalizedManyFaces( manyimgs ):
    detector = MTCNN( keep_all=True, device=device, post_process=False, select_largest=False)
    imgs = []
    for img in manyimgs:
        img_p = cv2.copyMakeBorder(img, 20, 20, 20, 20, cv2.BORDER_CONSTANT) 
        imgs.append( Image.fromarray( cv2.cvtColor( img_p, cv2.COLOR_GRAY2RGB ) ) )
    
    cropped_imgs = detector( imgs )
    
    sol = []
    for (img,ii) in zip(cropped_imgs,range(len(manyimgs))):
        if img is None:
            sol.append(manyimgs[ii])
        else:
            sol.append(cv2.resize(cv2.cvtColor( img.numpy()[0].T, cv2.COLOR_RGB2GRAY ).T,(224,224)))
    return sol    
    

In [5]:
CMFD_path = main_path + 'CMFD/'  #has 33 721 images
IMFD_path = main_path + 'IMFD/'  #has 33 588 images
max_subfolders = 2 #34

def create_dataframe( in_path, out_path, max_subfolders, crop_images=False):
    images = []
    for (folder, ii)  in zip(glob.glob(in_path+'*'),range(max_subfolders)):
        for image_path in glob.glob(folder+'/*.jpg'):
            images.append( process_img(image_path) )      

    if crop_images:
        images = ReturnLocalizedManyFaces(images)
    
    df = {'Image':[], 'LaofGau':[], 'SobelEdgeXY':[], 'CannyEdge':[]}
    for img in images:
        feature = feature_extraction(img)
        df['Image'].append(str(img.flatten().tolist()).replace(']','').replace('[',''))
        for key in feature.keys():
            df[key].append(str(feature[key].flatten().tolist()).replace(']','').replace('[',''))
    
    df = pd.DataFrame(df)
    if re.search ('CMFD', in_path):
        df['label'] = 0
        compression_opts = dict(method='zip',
                        archive_name='labels_0.csv')
        df.to_csv(out_path+'/'+'labels_0.csv.zip', compression=compression_opts, index=False)
    if re.search ('IMFD', in_path):
        df['label'] = 1
        compression_opts = dict(method='zip',
                        archive_name='labels_1.csv')
        df.to_csv(out_path+'/'+'labels_1.csv.zip', compression=compression_opts, index=False)
    
    return df        


In [6]:
if os.path.exists(CMFD_path+'/labels_0.csv.zip'):
    print("load CMFD csv file")
    df_C = pd.read_csv( CMFD_path+'/labels_0.csv.zip' )
else:
    print("create CMFD csv file")
    df_C = create_dataframe( CMFD_path, CMFD_path, max_subfolders)
    
if os.path.exists(IMFD_path+'/labels_1.csv.zip'):
    print("load IMFD csv file")
    df_I = pd.read_csv( IMFD_path+'/labels_1.csv.zip' )
else:
    print("create IMFD csv file")
    df_I = create_dataframe( IMFD_path, IMFD_path, max_subfolders)
    
df = pd.concat([df_I, df_C], axis=0, ignore_index=True).sample(frac=1)
df_C = None
df_I = None

load CMFD csv file
load IMFD csv file


## Training the models
Based in: 
  https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html and 
  https://github.com/sheldonsebastian/face_mask_detector/tree/main/Code 

In [7]:
from sklearn.metrics import accuracy_score
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchvision import models
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import TensorDataset
import torch.optim 
import time

In [8]:
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "resnet18"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 128  #bigger 8 is good for df.sample(frac=0.1) 

# Number of epochs to train for
num_epochs = 15

# Parameter for Adam Optimizer
LR = 0.001

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

The training algorithm

In [9]:
#@title
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for loader_dat  in dataloaders[phase]:
                inputs = loader_dat.inp
                labels = loader_dat.tgt
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data[:,1]) #torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, optimizer

The model to load/use 

In [10]:
#@title
# %% --------------------
def set_parameter_requires_grad(model, feature_extracting):
    # feature_extract_param = True means all layers frozen except the last user added layers
    # feature_extract_param = False means all layers unfrozen and entire network learns new weights
    # and biases
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
            

# %% --------------------initialize pretrained model & return input size desired by pretrained model
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet18":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "resnet50":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Sequential(nn.Dropout(0.3), nn.Linear(num_ftrs, num_classes))
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size



Data set Loading

In [11]:
#@title
def get_tensordataset(df, feature, transform=None):
    imgs = []
    labels = []
    for img_p, label in zip(df[feature],df['label']):
        img_p = np.array(img_p.split(', ')).astype(np.uint8).reshape( (224,224) )
        image = Image.fromarray( cv2.cvtColor( img_p, cv2.COLOR_GRAY2RGB ) )
        if transform:
            image = transform(image)
        else:
            generic_transformer = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
            image = generic_transformer(image)
        if label == 0:    
          labels.append( torch.Tensor( [ 1, 0 ] ) )
        if label == 1:
          labels.append( torch.Tensor( [ 0, 1 ] ) )
        imgs.append(image)
    
    a = torch.stack(imgs)
    b = torch.stack(labels) 
    
    return TensorDataset(a.view(a.shape),b.view(b.shape))

class SimpleCustomBatch:
    def __init__(self, data):
        transposed_data = list(zip(*data))
        self.inp = torch.stack(transposed_data[0], 0)
        self.tgt = torch.stack(transposed_data[1], 0)

    # custom memory pinning method on custom type
    def pin_memory(self):
        self.inp = self.inp.pin_memory()
        self.tgt = self.tgt.pin_memory()
        return self

def collate_wrapper(batch):
    return SimpleCustomBatch(batch)

In [12]:
#@title
def find_model_feature( feature, df_train, df_val ):
    if feature not in ['Image', 'LaofGau', 'SobelEdgeXY', 'CannyEdge'] :
        print('Bad feature name')
        return 

    # Initialize the model for this run
    model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

    # Print the model we just instantiated
    #print(model_ft)


    # Send the model to GPU
    model_ft = model_ft.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    

    # %% --------------------
    optimizer_ft = torch.optim.Adam(params_to_update, lr=LR)
    
    
    # Setup the loss fxn
    criterion = nn.CrossEntropyLoss()
    
    # Load data TODO
    # some augmentation
    train_transformer = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.ColorJitter(saturation=[0, 1]),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    # validation and holdout transformation is generic
    # normalization and resize
    generic_transformer = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    # create dataloader
    
        
    train_dataset = get_tensordataset(df_train, feature, transform=train_transformer)
    val_dataset = get_tensordataset(df_val, feature, transform=generic_transformer)
    
    train_dataloader = DataLoader(train_dataset,batch_size=batch_size, collate_fn=collate_wrapper, pin_memory=True )
    val_dataloader = DataLoader(val_dataset,batch_size=batch_size, collate_fn=collate_wrapper, pin_memory=True )

    # create dataloader dictionary
    dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

    # Train and evaluate
    print('Training feature: '+feature )
    model, hist, optim = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

    
    torch.cuda.empty_cache()

    return model, hist, optim

In [13]:
models_list = []
hist_list = []
df1 = df #df.sample(frac=0.1)
df_train, df2 = train_test_split(df1,test_size=0.4,random_state=2)
df_val, df_test = train_test_split(df2,test_size=0.5,random_state=2)
print('train size ',df_train.count()[0])
print('validation size ',df_val.count()[0])
print('test size ',df_test.count()[0])
feature_list = ['Image', 'LaofGau', 'SobelEdgeXY', 'CannyEdge']
for feature in feature_list:
    print('\n\n #####  Starting feature ', feature,' #####\n\n')
    sol1, sol2, sol3 = find_model_feature(feature,df_train,df_val)
    models_list.append(sol1)
    hist_list.append(sol2)

train size  2310
validation size  770
test size  771


 #####  Starting feature  Image  #####


Params to learn:
	 fc.weight
	 fc.bias
Training feature: Image
Epoch 0/14
----------
train Loss: 0.6590 Acc: 0.6100
val Loss: 0.5216 Acc: 0.7935

Epoch 1/14
----------
train Loss: 0.4441 Acc: 0.8463
val Loss: 0.3985 Acc: 0.8623

Epoch 2/14
----------
train Loss: 0.3522 Acc: 0.8831
val Loss: 0.3302 Acc: 0.8948

Epoch 3/14
----------
train Loss: 0.3036 Acc: 0.8991
val Loss: 0.2843 Acc: 0.9104

Epoch 4/14
----------
train Loss: 0.2738 Acc: 0.9061
val Loss: 0.2583 Acc: 0.9273

Epoch 5/14
----------
train Loss: 0.2528 Acc: 0.9113
val Loss: 0.2404 Acc: 0.9286

Epoch 6/14
----------
train Loss: 0.2371 Acc: 0.9173
val Loss: 0.2270 Acc: 0.9299

Epoch 7/14
----------
train Loss: 0.2247 Acc: 0.9208
val Loss: 0.2164 Acc: 0.9351

Epoch 8/14
----------
train Loss: 0.2145 Acc: 0.9229
val Loss: 0.2077 Acc: 0.9351

Epoch 9/14
----------
train Loss: 0.2059 Acc: 0.9238
val Loss: 0.2006 Acc: 0.9390

Epoch 10/14

## The union of models

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [14]:
def _find_new_coordinates( df_l, model_i, feature ):
    model_i.eval()
    acc_dataset = get_tensordataset( df_l, feature)    
    acc_dataloader = DataLoader( acc_dataset,batch_size=batch_size, collate_fn=collate_wrapper, pin_memory=True )

    pred = []
    actual = []
    coord = []

    # get all outputs
    for loader_dat  in acc_dataloader:
        inputs = loader_dat.inp
        labels = loader_dat.tgt
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            # logit
            predictions = model_i(inputs)

            # find index with max logit
            val, index = torch.max(predictions, 1)

            # append prediction
            pred.extend(index.tolist())

            # append actual
            actual.extend(labels.data[:,1].tolist())

            # append coordinates
            coord.extend( predictions.tolist() )

    accuracy = accuracy_score(actual, pred)
    f1 = f1_score(actual, pred, average='micro')
    cm = confusion_matrix(actual, pred)
    print("Accuracy::" + str(accuracy))
    print("F1 Score::" + str(f1))
    print("Confusion Matrix::" + str(cm))

    return coord, pred, actual

In [15]:
def find_new_coordinates( df, stringi ):
  print('\nCalculating in data set '+stringi.upper())
  coords = []
  preds = []
  actuals = []
  for ii in range(4):
    print('\nResults for ', feature_list[ii])
    coord, pred, actual = _find_new_coordinates( df, models_list[ii], feature_list[ii] )
    if ii == 0 :
      coords = coord
      preds = [ [ll] for ll in pred ]
      actuals = [ int(ll) for ll in actual ]
    else:
      for ii in range(len(coord)):
        coords[ii].extend(coord[ii])
        preds[ii].append(pred[ii])
   
  return {'coords_'+stringi:coords, 'preds_'+stringi:preds, 'actuals_'+stringi:actuals}        

In [16]:
test_coords = find_new_coordinates(df_test,'test')
train_coords = find_new_coordinates(df_train,'train')
val_coords = find_new_coordinates(df_val,'val')


Calculating in data set TEST

Results for  Image
Accuracy::0.9325551232166018
F1 Score::0.9325551232166018
Confusion Matrix::[[367  32]
 [ 20 352]]

Results for  LaofGau
Accuracy::0.8599221789883269
F1 Score::0.8599221789883269
Confusion Matrix::[[345  54]
 [ 54 318]]

Results for  SobelEdgeXY
Accuracy::0.8715953307392996
F1 Score::0.8715953307392996
Confusion Matrix::[[357  42]
 [ 57 315]]

Results for  CannyEdge
Accuracy::0.8573281452658884
F1 Score::0.8573281452658884
Confusion Matrix::[[351  48]
 [ 62 310]]

Calculating in data set TRAIN

Results for  Image
Accuracy::0.9337662337662338
F1 Score::0.9337662337662338
Confusion Matrix::[[1057   88]
 [  65 1100]]

Results for  LaofGau
Accuracy::0.8982683982683982
F1 Score::0.8982683982683982
Confusion Matrix::[[1030  115]
 [ 120 1045]]

Results for  SobelEdgeXY
Accuracy::0.90995670995671
F1 Score::0.90995670995671
Confusion Matrix::[[1052   93]
 [ 115 1050]]

Results for  CannyEdge
Accuracy::0.8748917748917749
F1 Score::0.8748917748917

In [17]:
def theFifthClassifier( test_coords, train_coords, val_coords ):
  df_test = np.array(test_coords['coords_test'])
  df_test_l = np.array(test_coords['actuals_test'])

  df_train = np.vstack( (np.array(train_coords['coords_train']), np.array(val_coords['coords_val']) ) )
  df_train_l = np.hstack( (np.array(train_coords['actuals_train']), np.array(val_coords['actuals_val']) ) )


  classif = KNeighborsClassifier(n_neighbors=8)
  def ClassifierFunction( ):
    return classif.fit( df_train, df_train_l )
  
  print( '######################' )
  print( 'Training' )
  classificated = ClassifierFunction( )
  print( '...' )                           
  print( 'Testing' )
  pred = classificated.predict(df_test)
  actual = df_test_l
  accuracy = accuracy_score(actual, pred)
  f1 = f1_score(actual, pred, average='micro')
  cm = confusion_matrix(actual, pred)
  print("The Fifht classifier")
  print("Accuracy::" + str(accuracy))
  print("F1 Score::" + str(f1))
  print("Confusion Matrix::" + str(cm))

  final_pred = []
  for ii in range(len(pred)):
    test_coords['preds_test'][ii].append(pred[ii])
    sol = np.sum(np.array(test_coords['preds_test'][ii]))
    if sol>=3:
      final_pred.append(1)
    else:
      final_pred.append(0)  

  accuracy = accuracy_score(actual, final_pred)
  f1 = f1_score(actual, final_pred, average='micro')
  cm = confusion_matrix(actual, final_pred)
  print("The Final prediction")
  print("Accuracy::" + str(accuracy))
  print("F1 Score::" + str(f1))
  print("Confusion Matrix::" + str(cm))

  return

In [18]:
theFifthClassifier( test_coords, train_coords, val_coords )


######################
Training
...
Testing
The Fifht classifier
Accuracy::0.9455252918287937
F1 Score::0.9455252918287937
Confusion Matrix::[[389  10]
 [ 32 340]]
The Final prediction
Accuracy::0.9416342412451362
F1 Score::0.9416342412451362
Confusion Matrix::[[387  12]
 [ 33 339]]
